<a href="https://colab.research.google.com/github/AAKAAASSHHH24/NLP-BASICS/blob/main/BERT_implementation_ktrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install ktrain

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 25.3 MB 1.4 MB/s 
     |████████████████████████████████| 981 kB 53.6 MB/s 
     |████████████████████████████████| 265 kB 70.7 MB/s 
     |████████████████████████████████| 3.8 MB 63.5 MB/s 
     |████████████████████████████████| 1.3 MB 67.3 MB/s 
     |████████████████████████████████| 468 kB 50.1 MB/s 
     |████████████████████████████████| 880 kB 69.8 MB/s 
     |████████████████████████████████| 7.6 MB 70.3 MB/s 
     |████████████████████████████████| 182 kB 77.2 MB/s 
  Created wheel for ktrain: filename=ktrain-0.32.3-py3-none-any.whl size=25313585 sha256=870ce9f0e9401be48cbb93047a3736b81d41013a103429e50074b6b28ec90503
  Stored in directory: /root/.cache/pip/wheels/c4/a0/4b/38365824295b72c59f340fce7927cdb5438d2705f5dbbe47b7
  Created wheel for keras-bert: filename=keras_bert-0.89.0-py3-none-any.whl size=33516 sha256=48fb75ca79881bc92a1c877f

In [4]:
import ktrain
from ktrain import text
from sklearn.datasets import fetch_20newsgroups

In [5]:
newsgroups_train = fetch_20newsgroups(subset = "train")

categories = newsgroups_train.target_names
categories


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [6]:
train = fetch_20newsgroups(subset = "train" , categories = categories, shuffle = True, random_state= 42)
test = fetch_20newsgroups(subset = "test" , categories = categories, shuffle = True, random_state= 42)

In [7]:
X_train, y_train, X_test, y_test = train.data, train.target, test.data, test.target

In [8]:
IDX = 0

print(X_train[IDX])
print(y_train[IDX])


From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.

Thanks,
- IL
   ---- brought to you by your neighborhood Lerxst ----





7


In [9]:
# PREPROCESSING
train_data, val_data, preprocess = text.texts_from_array(x_train= X_train,
                                             y_train= y_train,
                                             x_test= X_test,
                                             y_test= y_test,
                                             class_names= train.target_names,
                                             preprocess_mode='bert',
                                             maxlen = 350)

downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


task: text classification


In [10]:
model = text.text_classifier("bert", train_data = train_data, preproc = preprocess)

Is Multi-Label? False
maxlen is 350
done.


In [11]:
learner =  ktrain.get_learner(model, train_data = train_data, val_data = val_data, batch_size = 6)

In [12]:
LEARNING_RATE = 3e-5
EPOCHS = 4
learner.fit_onecycle(lr = LEARNING_RATE, epochs =EPOCHS)



begin training using onecycle policy with max lr of 3e-05...
Epoch 1/4
1886/1886 [==============================] - 1340s 700ms/step - loss: 1.2683 - accuracy: 0.6313 - val_loss: 0.6360 - val_accuracy: 0.8051
Epoch 2/4
1886/1886 [==============================] - 1321s 700ms/step - loss: 0.3718 - accuracy: 0.8889 - val_loss: 0.5886 - val_accuracy: 0.8363
Epoch 3/4
1886/1886 [==============================] - 1319s 699ms/step - loss: 0.1724 - accuracy: 0.9486 - val_loss: 0.5597 - val_accuracy: 0.8619
Epoch 4/4
1886/1886 [==============================] - 1319s 699ms/step - loss: 0.0473 - accuracy: 0.9860 - val_loss: 0.5202 - val_accuracy: 0.8728


In [13]:
predictor = ktrain.get_predictor(model, preproc = preprocess)

In [14]:
text_in = "Argentina won the world cup. Lionel Messi established himself as the greatest of all time"
predictor.predict(texts = text_in)

1/1 [==============================] - 4s 4s/step


'rec.sport.hockey'

In [15]:
trained_bert = "trained_bert/predictor_1"
predictor.save(trained_bert)

In [16]:
loaded_model = ktrain.load_predictor(trained_bert)
loaded_model.predict(text_in)

1/1 [==============================] - 4s 4s/step


'rec.sport.hockey'